# Categorical data

Categorical data is a collection of information that is divided into groups. I.e, if an organisation or agency is trying to get a biodata of its employees, the resulting data is referred to as categorical. This data is called categorical because it may be grouped according to the variables present in the biodata such as sex, state of residence, etc.

## Types of categorical data

Categorical data often includes values and observations that can be categorized or grouped. Bar graphs and pie charts are the best ways to show this data. More specifically, there are two kinds of categorical data:

- Nominal Data
- Ordinal Data


# Load and processing data


In [24]:
import pandas as pd

In [25]:
dataset = pd.read_csv("./datasets/Data.csv")
dataset

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes
5,France,35.0,58000.0,Yes
6,Spain,NaN,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


In [26]:
x = dataset.iloc[:,:-1].values
y = dataset.iloc[:,3].values # Prediction column

In [27]:
# Rows
x

array([['France', 44.0, 72000.0],
       ['Spain', 27.0, 48000.0],
       ['Germany', 30.0, 54000.0],
       ['Spain', 38.0, 61000.0],
       ['Germany', 40.0, nan],
       ['France', 35.0, 58000.0],
       ['Spain', nan, 52000.0],
       ['France', 48.0, 79000.0],
       ['Germany', 50.0, 83000.0],
       ['France', 37.0, 67000.0]], dtype=object)

In [28]:
# Cols
y

array(['No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes'],
      dtype=object)

# Manage missing data


In [29]:
from sklearn.impute import SimpleImputer
import numpy as np

In [30]:
dataset['Salary'].isna().sum()

1

In [31]:
dataset['Salary']

0    72000.0
1    48000.0
2    54000.0
3    61000.0
4        NaN
5    58000.0
6    52000.0
7    79000.0
8    83000.0
9    67000.0
Name: Salary, dtype: float64

In [32]:
dataset['Age']

0    44.0
1    27.0
2    30.0
3    38.0
4    40.0
5    35.0
6     NaN
7    48.0
8    50.0
9    37.0
Name: Age, dtype: float64

In [33]:
# Define directrix to work with
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# Use the directrix to adjust the missing data
imp = imp.fit(x[:,1:3])
# Override set of data. All the rows, cols 1 up to 3
x[:,1:3] = imp.transform(x[:,1:3])
x

array([['France', 44.0, 72000.0],
       ['Spain', 27.0, 48000.0],
       ['Germany', 30.0, 54000.0],
       ['Spain', 38.0, 61000.0],
       ['Germany', 40.0, 63777.77777777778],
       ['France', 35.0, 58000.0],
       ['Spain', 38.77777777777778, 52000.0],
       ['France', 48.0, 79000.0],
       ['Germany', 50.0, 83000.0],
       ['France', 37.0, 67000.0]], dtype=object)

# Categorize data


In [34]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [35]:
# LabelEncoder version
""" from sklearn.preprocessing import LabelEncoder
# Label encoder column 0 'Country'

le1 = LabelEncoder()
x[:,0] = le1.fit_transform(x[:,0])
x """

" from sklearn.preprocessing import LabelEncoder\n# Label encoder column 0 'Country'\n\nle1 = LabelEncoder()\nx[:,0] = le1.fit_transform(x[:,0])\nx "

In [36]:
# Categorize country column from x
country_col_transform = ColumnTransformer(
    [
        (
            'one_hot_encoder',
            OneHotEncoder(categories='auto'),
            [0]
        )
    ],
    remainder='passthrough'
)

x = np.array(country_col_transform.fit_transform(x), dtype=np.int64)

pd.DataFrame(x, columns=["France", "Spain", "Germany", "Age", "Salary"])

,France,Spain,Germany,Age,Salary
0,1,0,0,44,72000
1,0,0,1,27,48000
2,0,1,0,30,54000
3,0,0,1,38,61000
4,0,1,0,40,63777
5,1,0,0,35,58000
6,0,0,1,38,52000
7,1,0,0,48,79000
8,0,1,0,50,83000
9,1,0,0,37,67000


In [37]:
# Categorize 'Purchased' column from y
purchased_col_transform = ColumnTransformer(
    [
        (
            'one_hot_encoder',
            OneHotEncoder(categories='auto'),
            [0]
        )
    ]
)

y = np.array(purchased_col_transform.fit_transform(x), dtype=np.int64)
pd.DataFrame(y, columns=["Yes", "No"])

,Yes,No
0,0,1
1,1,0
2,1,0
3,1,0
4,1,0
5,0,1
6,1,0
7,0,1
8,1,0
9,0,1


# Split dataset up into training and testing sets

The goal is creating a model that generalize correctly new entries (new data). Training test is useful and plays the role as proxy for new data.

- Training set: subset for training a model
- Testing set: subset for testing the trained model

Things to keep in mind

- Training subset should be big enough to produce significative results from statistic view
- Do not choose a testing subset with different features than the training set

`NOTE: DO NOT USE TESTING DATA FOR TRAINING.` If you see amazingly results on your evaluation metrics, it‘s possible you being training the testing subset.

Example: To Find a 99% of precision on testing and training subset because of there are duplicate entries as in the training subset as the testing subset.

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
# x data matrix; y variable to forecast
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=0)

In [40]:
# 80% for training
x_train

array([[    0,     1,     0,    40, 63777],
       [    1,     0,     0,    37, 67000],
       [    0,     0,     1,    27, 48000],
       [    0,     0,     1,    38, 52000],
       [    1,     0,     0,    48, 79000],
       [    0,     0,     1,    38, 61000],
       [    1,     0,     0,    44, 72000],
       [    1,     0,     0,    35, 58000]], dtype=int64)

In [41]:
y_train

array([[1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1]], dtype=int64)

In [42]:
# 20 % for testing
x_test

array([[    0,     1,     0,    30, 54000],
       [    0,     1,     0,    50, 83000]], dtype=int64)

In [43]:
y_test

array([[1, 0],
       [1, 0]], dtype=int64)

# Scaling and normalization data 

Difference:

- in scaling, you're changing the range of your data.
- in normalization, you're changing the shape of the distribution of your data.

Scaling: 

`This means that you're transforming your data so that it fits within a specific scale, like 0-100 or 0-1.` You want to scale data when you're using methods based on measures of how far apart data points are, like support vector machines (SVM) or k-nearest neighbors (KNN). With these algorithms, a change of "1" in any numeric feature is given the same importance.

``Data scaling is the process of making sure that all the values in a dataset are within a certain range,``

Normalization:

Normalization is a more radical transformation. `The point of normalization is to change your observations so that they can be described as a normal distribution.`

`Normal distribution: Also known as the "bell curve"`, this is a specific statistical `distribution where a roughly equal observations fall above and below the mean`, the mean and the median are the same, and there are more observations closer to the mean. The normal distribution is also known as the Gaussian distribution.`

``Data normalization is the process of making sure that all the values in a dataset are close to each other, such as having a mean of 0.``


Let’s consider X as the column and x every item:

$$ N = {x - min(X)\over max(X) - min(X)} $$


[Source info](https://www.kaggle.com/code/alexisbcook/scaling-and-normalization)



In [44]:
from sklearn.preprocessing import StandardScaler
# create scaler 
scaler_X = StandardScaler()
# Replace data using the x_train
x_train = scaler_X.fit_transform(x_train)
# Using the same scale for test set
x_test = scaler_X.transform(x_test)

In [45]:
pd.DataFrame(x_train, columns=["France", "Spain", "Germany", "Age", "Salary"])

,France,Spain,Germany,Age,Salary
0,-1.0,2.645751,-0.774597,0.279780,0.123744
1,1.0,-0.377964,-0.774597,-0.236737,0.461767
2,-1.0,-0.377964,1.290994,-1.958462,-1.530925
3,-1.0,-0.377964,1.290994,-0.064565,-1.111411
4,1.0,-0.377964,-0.774597,1.657160,1.720310
5,-1.0,-0.377964,1.290994,-0.064565,-0.167504
6,1.0,-0.377964,-0.774597,0.968470,0.986160
7,1.0,-0.377964,-0.774597,-0.581082,-0.482140


In [46]:
pd.DataFrame(x_test, columns=["France", "Spain", "Germany", "Age", "Salary"])

,France,Spain,Germany,Age,Salary
0,-1.0,2.645751,-0.774597,-1.441944,-0.901654
1,-1.0,2.645751,-0.774597,2.001505,2.139824
